### BP Smoky Mountain Data Challenge

#### Author: Max Grossman (max.grossman@bp.com)

This notebook is intended to be an illustrative example of loading the realizations and gathers provided as part of the BP problem in the 2020 Smoky Mountain Data Challenge. This notebook relies on data files kept under the small_dataset/ folder provided with the data challenge. Please ensure that the folder is downloaded and unzipped, and placed in the same directory as this notebook.

This notebook requires that the numpy, matplotlib, pillow, and segpy packages be installed in the current Python environment. numpy, matplotlib, and pillow may be installed using pip or conda. segpy is only available from pip (i.e. 'pip install segpy').

In [ ]:
# Install a pip package in the current Jupyter kernel
import segpy
from segpy.reader import create_reader
from segpy.writer import write_segy

import math
import matplotlib as plot
import matplotlib.pyplot
%matplotlib inline
import numpy as np
import pandas as pd
from PIL import Image

import os
import sys

import cv2
import pickle

import sklearn
from skimage import data, img_as_float
from skimage.measure import compare_ssim as ssim

In [ ]:
"""
Utility for pulling information out of SEGY header and printing it for the user.
"""
def print_segy_info(filename):
    with open(filename, 'rb') as segy_in_file:
        segy_reader = create_reader(segy_in_file, endian='>')

        print("Filename:             ", segy_reader.filename)
        print("SEG Y revision:       ", segy_reader.revision)
        print("Number of traces:     ", segy_reader.num_traces())
        print("Data format:          ",
              segy_reader.data_sample_format_description)
        print("Dimensionality:       ", segy_reader.dimensionality)

        print()
        print("=== BEGIN TEXTUAL REEL HEADER ===")
        for line in segy_reader.textual_reel_header:
            if len(line.strip()) > 0:
                print(line)
        print("=== END TEXTUAL REEL HEADER ===")

In [ ]:
"""
The provided files ending with a .stack.segy suffix store realizations of the subsurface
using some plausible (but not perfect) velocity model.

These realizations are 2D, with a depth (z) dimension of 400 and a horizontal (x) dimension
of 1058. This information can be seen in the SEGY header displayed below.
"""
print_segy_info("C:/Users/19174/Desktop/BP HPC/large_dataset_only_realizations/img_1.stack.segy")


In [ ]:
"""
Example of loading a single 2D realization from a single file into a 2D numpy array, with x
as your leading dimension and z as your innermost dimension. We know the x and z dimensions
(1058 and 400) from the SEGY header printed by the cell above.
"""
REALIZATIONS = 1000
deviations = np.zeros((1058, 400))
means = np.zeros((1058, 400))
totals = np.zeros((1058, 400))
PATH = "C:/Users/19174/Desktop/BP HPC/large_dataset_only_realizations"
                    

In [ ]:
import random
random_std = []
for file in os.listdir(PATH):
    if file.endswith(".stack.segy"):
        print(file)
        path = "C:/Users/19174/Desktop/BP HPC/large_dataset_only_realizations/"+ file
        with open(path, 'rb') as segy_in_file:
            segy_reader = create_reader(segy_in_file, endian='>')
            for trace_index in segy_reader.trace_indexes():
                data = segy_reader.trace_samples(trace_index)
                for i in range(len(data)):
                    means[trace_index, i] += data[i]/REALIZATIONS #Collecting means, storing in array
                    temp = random.randint(0, 423200)
                    if temp <= 100:
                        x = float(data[i])
                        random_std.append(x)
                        

In [ ]:
df = pd.Series(random_std)
df.plot(kind='hist', figsize = (25, 6), bins=200, title = "Distribution of Random Pixel Values across Realizations")

In [ ]:
#Probably better way than to rerun over files :( 
for file in os.listdir(PATH):
    if file.endswith(".stack.segy"):
        print(file)
        path = "C:/Users/19174/Desktop/BP HPC/large_dataset_only_realizations/"+ file
        with open(path, 'rb') as segy_in_file:
            segy_reader = create_reader(segy_in_file, endian='>')
            for trace_index in segy_reader.trace_indexes():
                data = segy_reader.trace_samples(trace_index)
                for i in range(len(data)):
                    totals[trace_index, i] += (data[i] - means[trace_index, i]) ** 2


In [ ]:

#print("Deviations: Rows", len(deviations), "Cols ", len(deviations[0]))
#print("Cell - Mean: Rows", len(totals) , "Cols ", len(totals[0]))
#print("Mean: Rows", len(means), "Cols", len(means[0]))

for i in range(len(deviations)):
    for j in range(len(deviations[0])):

        deviations[i][j] = math.sqrt(totals[i][j]/REALIZATIONS)


In [ ]:
deviations.min(), deviations.max()

In [ ]:
#only run once for large data
with open('large_deviation.pickle', 'wb') as f:
    pickle.dump(deviations, f)

In [ ]:
#Use this to access large st deviation matrix
with open('large_deviation.pickle', 'rb') as f:
     deviations= pickle.load(f) #deviations

In [ ]:
#Make copies of deviations to avoid manipulating true data
flat_deviations = deviations.flatten()
print("Max STD: ",deviations.max())
df = pd.Series(flat_deviations)
df.plot(kind='hist', figsize = (25, 6), bins=200, title = "Distribution of Standard Deviations")

In [ ]:
SAMPLE_REAL = "C:/Users/19174/Desktop/BP HPC/large_dataset_only_realizations/img_5.stack.segy"
sample_arr = np.zeros([1058, 400, 3], dtype=np.uint8)
gray_sample_arr = np.zeros((1058, 400))
with open(SAMPLE_REAL, 'rb') as segy_in_file:
    segy_reader = create_reader(segy_in_file, endian='>')
    for trace_index in segy_reader.trace_indexes():
        data = segy_reader.trace_samples(trace_index)
        for i in range(len(data)):
            gray_sample_arr[trace_index, i] = data[i]
with open(SAMPLE_REAL, 'rb') as segy_in_file:
    segy_reader = create_reader(segy_in_file, endian='>')
    for trace_index in segy_reader.trace_indexes():
        data = segy_reader.trace_samples(trace_index)
        for i in range(len(data)):
            sample_arr[trace_index, i] = [data[i], 0, 0] #Collecting data for sample_arr

min_arr = gray_sample_arr.min()
max_arr = gray_sample_arr.max()
gray_sample_arr = (gray_sample_arr - min_arr) / (max_arr - min_arr) #Normalizes all values to 0 - 1
gray_sample_arr = gray_sample_arr * 255
for i in range(len(gray_sample_arr)):
    for j in range(len(gray_sample_arr[0])):
        sample_arr[i][j] = [gray_sample_arr[i][j], gray_sample_arr[i][j], gray_sample_arr[i][j]]

In [ ]:
THRESHOLD = 2.0 #If STD > THRESHOLD plot
deviations_copy = deviations.copy()
max_dev = deviations_copy.max()
for i in range(len(deviations_copy)):
    for j in range(len(deviations_copy[0])):
        if deviations_copy[i][j] >= THRESHOLD:
            color_intensity = sample_arr[i][j][0]
            sample_arr[i][j] = [color_intensity, 0, 0]

im = Image.fromarray(sample_arr)
im = im.convert('RGB')
transposed  = im.transpose(Image.ROTATE_270)


transposed

In [ ]:
gray_sample_arr = gray_sample_arr.transpose() #Reformat 
min_arr = gray_sample_arr.min()
max_arr = gray_sample_arr.max()
gray_sample_arr = (gray_sample_arr - min_arr) / (max_arr - min_arr) #Normalizes all values to 0 - 1
gray_sample_arr = gray_sample_arr * 255  #Multiply everything to RGB Scale
im = Image.fromarray(gray_sample_arr)
im = im.convert('RGB')

im

In [ ]:
gray_sample_arr = np.zeros((1058, 400))
with open("C:/Users/19174/Desktop/BP HPC/small_dataset/small_dataset/img_1.stack.segy", 'rb') as segy_in_file:
    segy_reader = create_reader(segy_in_file, endian='>')
    for trace_index in segy_reader.trace_indexes():
        data = segy_reader.trace_samples(trace_index)
        for i in range(len(data)):
            gray_sample_arr[trace_index, i] = data[i]
gray_sample_arr = gray_sample_arr.transpose() #Reformat 
min_arr = gray_sample_arr.min()
max_arr = gray_sample_arr.max()
gray_sample_arr = (gray_sample_arr - min_arr) / (max_arr - min_arr) #Normalizes all values to 0 - 1
gray_sample_arr = gray_sample_arr * 255  #Multiply everything to RGB Scale
im = Image.fromarray(gray_sample_arr)
im = im.convert('RGB')

im

In [ ]:
gray_sample_arr = np.zeros((1058, 400))
with open("C:/Users/19174/Desktop/BP HPC/small_dataset/small_dataset/img_2.stack.segy", 'rb') as segy_in_file:
    segy_reader = create_reader(segy_in_file, endian='>')
    for trace_index in segy_reader.trace_indexes():
        data = segy_reader.trace_samples(trace_index)
        for i in range(len(data)):
            gray_sample_arr[trace_index, i] = data[i]
gray_sample_arr = gray_sample_arr.transpose() #Reformat 
min_arr = gray_sample_arr.min()
max_arr = gray_sample_arr.max()
gray_sample_arr = (gray_sample_arr - min_arr) / (max_arr - min_arr) #Normalizes all values to 0 - 1
gray_sample_arr = gray_sample_arr * 255  #Multiply everything to RGB Scale
im = Image.fromarray(gray_sample_arr)
im = im.convert('RGB')

im

In [ ]:
#Template of breaking up a single realization into 23X20 tiles

first_real = "C:/Users/19174/Desktop/BP HPC/small_dataset/small_dataset/img_1.stack.segy"
hori_size = 46
vert_size = 20
ans = []
for i in range(int(1058/ hori_size)):
    test = []
    for j in range(int(400 / vert_size)):
        test.append([])
    ans.append(test)
REALS = 39
for _ in range(1, REALS + 1):
    first_real = "C:/Users/19174/Desktop/BP HPC/small_dataset/small_dataset/img_" + str(_) +  ".stack.segy"
    for temp in range(_ + 1, REALS + 1):
        second_real = "C:/Users/19174/Desktop/BP HPC/small_dataset/small_dataset/img_" + str(temp) + ".stack.segy"
        print("First File: ",first_real)
        print("Second File: ", second_real)
        x_index = 0
        y_index = 0
        with open(first_real, 'rb') as first_file, open(second_real, 'rb') as second_file:
            first_reader = create_reader(first_file, endian = '>')
            second_reader = create_reader(second_file, endian = '>')
            #i and j are top left of partition array
            for i in range(0, 1058, hori_size):
                for j in range(0, 400, vert_size):
                    first_map = np.zeros((hori_size,vert_size))
                    second_map = np.zeros((hori_size,vert_size))
                    #k and m responsible for partition array
                    for k in range(i, i + hori_size):
                        data1 = first_reader.trace_samples(k)
                        data2 = second_reader.trace_samples(k)
                        for m in range(j, j + vert_size): #Loop sets matrix to these elements
                            x = k% hori_size
                            y = m % vert_size
                            first_map[x, y] = data1[m]
                            second_map[x, y] = data2[m]
                    if first_map.max() != 0 and first_map.min() != 0:
                        first_map = first_map.transpose() #Reformat 
                        min_arr = first_map.min()
                        max_arr = first_map.max()
                        first_map = (first_map - min_arr) / (max_arr - min_arr) #Normalizes all values to 0 - 1
                        first_map = first_map * 255  #Multiply everything to RGB Scale
                        im1 = Image.fromarray(first_map)
                        im1 = im1.convert('RGB')
                    else:
                        #white image
                        im1 = Image.fromarray(first_map)
                        im1 = im1.convert('RGB')
                    if second_map.max() != 0 and second_map.min() != 0:
                        second_map = second_map.transpose() #Reformat 
                        min_arr = second_map.min()
                        max_arr = second_map.max()
                        second_map = (second_map - min_arr) / (max_arr - min_arr) #Normalizes all values to 0 - 1
                        second_map = second_map * 255  #Multiply everything to RGB Scale
                        im2 = Image.fromarray(second_map)
                        im2 = im2.convert('RGB')
                    else:
                        im2 = Image.fromarray(second_map)
                        im2 = im2.convert('RGB')
                    im1 = im1.save("image_1.jpg")
                    im2 = im2.save("image_2.jpg")
                    img1 = cv2.imread('image_1.jpg', cv2.IMREAD_UNCHANGED)
                    img2 = cv2.imread('image_2.jpg', cv2.IMREAD_UNCHANGED)
                    #All ssim seem to equal 1.0 or very close
                    s = ssim(img1, img2, multichannel= True)
                    ans[x_index][y_index].append(s)
                    y_index += 1
                x_index += 1
                y_index = 0
with open('ssim_map.pickle', 'wb') as f:
    pickle.dump(ans, f)
for i in range(len(ans)):
    for j in range(len(ans[0])):
        ans[i][j] = [np.mean(ans[i][j])]
print(ans)


In [ ]:
"""
Render the loaded realization by transposing it (so that the image is oriented intuitively) and normalizing.
"""
deviations_copy = deviations.copy()
deviations_copy = deviations_copy.transpose()
min_arr = deviations_copy.min()
max_arr = deviations_copy.max()
deviations_copy = (deviations_copy - min_arr) / (max_arr - min_arr)
deviations_copy = deviations_copy * 255

im = Image.fromarray(deviations_copy)
im = im.convert('RGB')
im

In [ ]:
"""
The provided files ending with a .gather.segy suffix store offset pair gathers generated
using some plausible (but not perfect) velocity model. Each offset pair gather file corresponds
with a single realization.

These gather files are 3D, with 39 offset pairs in the survey. The z and x dimensions are the
same as the realizations, with an added dimension indicating the offset pair. This information
can be seen in the SEGY header displayed below.
"""
print_segy_info('Desktop/BP HPC/small_dataset/small_dataset/img_3.gather.segy')

In [ ]:
"""
Load gathers into a 3D numpy array with dimensions (x, offsets, z).
"""
arr = np.zeros((1058, 39, 400))
with open('Desktop/BP HPC/small_dataset/small_dataset/img_3.gather.segy', 'rb') as segy_in_file:
    segy_reader = create_reader(segy_in_file, endian='>')
    
    count = 0
    for trace_index in segy_reader.trace_indexes():
        data = segy_reader.trace_samples(trace_index)
        for i in range(len(data)):
            arr[int(trace_index / 39), trace_index % 39, i] = data[i]


In [ ]:
arr.min(), arr.max()

In [ ]:
"""
Render the loaded gathers.
"""

nrows = 1
ncols = 10
f, axarr = matplotlib.pyplot.subplots(nrows, ncols, figsize = (30, 25))

for c in range(ncols):
    # Pull out all offset pairs for a given x
    sub_arr = arr[c, :, :]
    sub_arr = sub_arr.transpose()
    min_arr = sub_arr.min()
    max_arr = sub_arr.max()
    sub_arr = (sub_arr - min_arr) / (max_arr - min_arr)
    sub_arr = sub_arr * 255
    
    axarr[c].imshow(sub_arr, cmap = 'gray')
    
    title = str(c)
    axarr[c].set_title(title.strip())